In [10]:
%load_ext autoreload
%autoreload 2

# Control figure size
figsize=(14, 4)

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import random
import pandas as pd
from util import util
import numpy as np
import os
data_folder = os.path.join('.', 'files/data')
file_name = "Prepared_data_E0"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Regression Filter

### Load data

In [11]:
data = util.load_data(data_folder, file_name)
data = data[data['Div'] == 'E0']

data


,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,Season,ELO diff,Diff_goals_scored,Diff_goals_conceded,...,Diff_shots_on_target_attempted,Diff_shots_on_target_allowed,Diff_shots_attempted,Diff_shots_allowed,Diff_corners_awarded,Diff_corners_conceded,Diff_fouls_commited,Diff_fouls_suffered,Diff_yellow_cards,Diff_red_cards
0,E0,2005-08-13,Aston Villa,Bolton,2.0,2.0,0506,0.000000,0,0,...,0,0,0,0,0,0,0,0,0,0
1,E0,2005-08-13,Everton,Man United,0.0,2.0,0506,0.000000,0,0,...,0,0,0,0,0,0,0,0,0,0
2,E0,2005-08-13,Fulham,Birmingham,0.0,0.0,0506,0.000000,0,0,...,0,0,0,0,0,0,0,0,0,0
3,E0,2005-08-13,Man City,West Brom,0.0,0.0,0506,0.000000,0,0,...,0,0,0,0,0,0,0,0,0,0
4,E0,2005-08-13,Middlesbrough,Liverpool,0.0,0.0,0506,0.000000,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7373,E0,2024-12-14,Nott'm Forest,Aston Villa,2.0,1.0,2425,-74.054796,-1,3,...,-12,10,-26,20,-29,21,-13,-24,-1,0
7374,E0,2024-12-15,Brighton,Crystal Palace,1.0,3.0,2425,48.033586,2,1,...,2,0,0,4,-5,-12,10,-1,0,0
7375,E0,2024-12-15,Man City,Man United,1.0,2.0,2425,196.417030,-4,4,...,3,2,18,11,22,-9,5,-4,7,1
7376,E0,2024-12-15,Chelsea,Brentford,2.0,1.0,2425,168.028475,3,-2,...,16,-7,39,-39,10,-13,18,5,3,-1


In [12]:
data.drop(columns="Div", inplace=True)
from sklearn.preprocessing import LabelEncoder
data = data.copy()
label_encoder = LabelEncoder()

#Convert Date to numerial values
data["Year"] = data["Date"].dt.year
data["Month"] = data["Date"].dt.month
data["Day"] = data["Date"].dt.day
data["DayOfWeek"] = data[
    "Date"
].dt.dayofweek  # Optional

# Drop the original Date column as it’s no longer needed
data = data.drop(columns=["Date"])

In [13]:
'''data = pd.get_dummies(
    data, columns=["HomeTeam", "AwayTeam"]
)'''

'data = pd.get_dummies(\n    data, columns=["HomeTeam", "AwayTeam"]\n)'

In [14]:
data["Outcome"] = data.apply(
    lambda row: (
        row["FTHG"] - row["FTAG"]
    ),
    axis=1,
)

In [15]:
data

,HomeTeam,AwayTeam,FTHG,FTAG,Season,ELO diff,Diff_goals_scored,Diff_goals_conceded,Diff_goal_diff,Diff_points,...,Diff_corners_conceded,Diff_fouls_commited,Diff_fouls_suffered,Diff_yellow_cards,Diff_red_cards,Year,Month,Day,DayOfWeek,Outcome
0,Aston Villa,Bolton,2.0,2.0,0506,0.000000,0,0,0,0,...,0,0,0,0,0,2005,8,13,5,0.0
1,Everton,Man United,0.0,2.0,0506,0.000000,0,0,0,0,...,0,0,0,0,0,2005,8,13,5,-2.0
2,Fulham,Birmingham,0.0,0.0,0506,0.000000,0,0,0,0,...,0,0,0,0,0,2005,8,13,5,0.0
3,Man City,West Brom,0.0,0.0,0506,0.000000,0,0,0,0,...,0,0,0,0,0,2005,8,13,5,0.0
4,Middlesbrough,Liverpool,0.0,0.0,0506,0.000000,0,0,0,0,...,0,0,0,0,0,2005,8,13,5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7373,Nott'm Forest,Aston Villa,2.0,1.0,2425,-74.054796,-1,3,-4,-1,...,21,-13,-24,-1,0,2024,12,14,5,1.0
7374,Brighton,Crystal Palace,1.0,3.0,2425,48.033586,2,1,1,2,...,-12,10,-1,0,0,2024,12,15,6,-2.0
7375,Man City,Man United,1.0,2.0,2425,196.417030,-4,4,-8,-3,...,-9,5,-4,7,1,2024,12,15,6,-1.0
7376,Chelsea,Brentford,2.0,1.0,2425,168.028475,3,-2,5,3,...,-13,18,5,3,-1,2024,12,15,6,1.0


In [16]:
print("Outcome distribution:")
print(data["Outcome"].value_counts())

Outcome distribution:
Outcome
 0.0    1786
 1.0    1566
-1.0    1170
 2.0    1026
-2.0     606
 3.0     475
-3.0     280
 4.0     200
-4.0     102
 5.0      83
-5.0      35
 6.0      23
-6.0       9
 7.0       7
 8.0       5
 9.0       2
-9.0       1
-7.0       1
-8.0       1
Name: count, dtype: int64


In [17]:
X = data.copy().drop(
    columns=["Outcome", "FTHG", "FTAG", "Season", 'HomeTeam', 'AwayTeam']
)  # Drop columns not needed for prediction
y = data.copy()["Outcome"]
print(X.columns)

from sklearn.model_selection import train_test_split

# Split data into training and test sets (e.g., 80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

Index(['ELO diff', 'Diff_goals_scored', 'Diff_goals_conceded',
       'Diff_goal_diff', 'Diff_points', 'Diff_change_in_ELO',
       'Diff_opposition_mean_ELO', 'Diff_shots_on_target_attempted',
       'Diff_shots_on_target_allowed', 'Diff_shots_attempted',
       'Diff_shots_allowed', 'Diff_corners_awarded', 'Diff_corners_conceded',
       'Diff_fouls_commited', 'Diff_fouls_suffered', 'Diff_yellow_cards',
       'Diff_red_cards', 'Year', 'Month', 'Day', 'DayOfWeek'],
      dtype='object')


### Random forest Regressor instead of classifier


In [12]:
rf = RandomForestRegressor(n_estimators=200, random_state=42)
rf.fit(X_train,y_train)



RandomForestRegressor(n_estimators=200, random_state=42)

In [13]:
predictions = rf.predict(X_test)

In [14]:
mse = mean_squared_error(y_test, predictions)
print(f'Mean Squared Error: {mse}')

Mean Squared Error: 2.848441243224932


### Turning expected goal difference into categories (Win,Loss,Draw)

In [15]:
thresholds = {"hw": 1, "hl": -1}

def categorize_preds(pred_arr, hw, hl):
    categories = np.where(pred_arr > hw, 1, 
                          np.where((pred_arr <= hw) & (pred_arr > hl), 0, 
                                   -1))
    return np.array(categories)


def categorize_goal_diff(y_test):
    categories = np.where(y_test > 0, 1, 
                        np.where((y_test == 0 ), 0, 
                                -1))
    return np.array(categories)



In [16]:
from sklearn.metrics import classification_report

categorized_preds = categorize_preds(predictions, thresholds['hw'], thresholds['hl'])
categorized_goal_diff = categorize_goal_diff(y_test)

report = classification_report(categorized_goal_diff, categorized_preds)
print(report)


              precision    recall  f1-score   support

          -1       0.67      0.14      0.23       456
           0       0.26      0.76      0.39       347
           1       0.70      0.36      0.48       673

    accuracy                           0.39      1476
   macro avg       0.54      0.42      0.37      1476
weighted avg       0.58      0.39      0.38      1476



### Filter out matches we are uncertain about (Where expected goal diff is within [-1,1])

In [17]:


def remove_uncertain(predictions, targets):
    # Create a boolean mask where predictions is not 0
    mask = predictions != 0
    
    # Use the mask to filter both predictions and targets
    filtered_predictions = predictions[mask]
    filtered_targets = targets[mask]
    
    return filtered_predictions, filtered_targets

filtered_predictions, filtered_targets = remove_uncertain(categorized_preds, categorized_goal_diff)


### Get much better results by filtering out

In [18]:
report = classification_report(filtered_predictions, filtered_targets)
print(report)




              precision    recall  f1-score   support

          -1       0.62      0.67      0.64        96
           0       0.00      0.00      0.00         0
           1       0.94      0.70      0.80       351

    accuracy                           0.69       447
   macro avg       0.52      0.45      0.48       447
weighted avg       0.87      0.69      0.77       447



/Users/heiken/CVSnacks/BetMaximizer API/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/heiken/CVSnacks/BetMaximizer API/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/heiken/CVSnacks/BetMaximizer API/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.cap

### Now we only predict 30% of the matches, but with higher confidence

In [19]:
print(f'Bet Accceptance Rate: {len(filtered_predictions)/len(categorized_preds)}')

Bet Accceptance Rate: 0.30284552845528456
